In [ ]:
%pip install pandas
%pip install numpy 

In [1]:
import pandas as pd
import numpy as np
import re
from collections import Counter

In [2]:
df_hbo = pd.read_csv("C:/Users/nise_/OneDrive/Documentos/Digital College/Python/DA_18/streaming/Dados/data_hbo_max.csv")

In [5]:
df_hbo

,title,type,genres,releaseYear,imdbId,imdbAverageRating,imdbNumVotes,availableCountries
0,Kill Bill: Vol. 1,movie,"Action, Crime, Thriller",2003.0,tt0266697,8.2,1251651.0,NaN
1,Jarhead,movie,"Biography, Drama, War",2005.0,tt0418763,7.0,215816.0,NaN
2,Unforgiven,movie,"Drama, Western",1992.0,tt0105695,8.2,454694.0,NaN
3,Eternal Sunshine of the Spotless Mind,movie,"Drama, Romance, Sci-Fi",2004.0,tt0338013,8.3,1137227.0,NaN
4,2001: A Space Odyssey,movie,"Adventure, Sci-Fi",1968.0,tt0062622,8.3,754036.0,NaN
...,...,...,...,...,...,...,...,...
10221,The House That Dragons Built,tv,Documentary,2022.0,tt26761151,8.1,235.0,NaN
10222,NaN,tv,Reality,2025.0,NaN,NaN,NaN,NaN
10223,Toxic,tv,Documentary,2025.0,tt36495586,6.2,79.0,NaN
10224,The Bakersfield 3: A Tale of Murder and Mother...,tv,Documentary,2025.0,tt36926665,6.3,112.0,NaN


In [7]:
df_hbo.describe()

,releaseYear,imdbAverageRating,imdbNumVotes
count,10198.000000,9089.000000,9.089000e+03
mean,2012.058051,6.693157,5.284668e+04
std,17.017881,1.041654,1.653684e+05
min,1898.000000,1.200000,5.000000e+00
25%,2010.000000,6.100000,1.960000e+02
50%,2018.000000,6.700000,1.354000e+03
75%,2022.000000,7.400000,2.058300e+04
max,2027.000000,9.800000,3.046940e+06


In [8]:
df_hbo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10226 entries, 0 to 10225
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               9576 non-null   object 
 1   type                10226 non-null  object 
 2   genres              9904 non-null   object 
 3   releaseYear         10198 non-null  float64
 4   imdbId              9217 non-null   object 
 5   imdbAverageRating   9089 non-null   float64
 6   imdbNumVotes        9089 non-null   float64
 7   availableCountries  68 non-null     object 
dtypes: float64(3), object(5)
memory usage: 639.3+ KB


In [9]:
df_hbo.isnull().sum()

title                   650
type                      0
genres                  322
releaseYear              28
imdbId                 1009
imdbAverageRating      1137
imdbNumVotes           1137
availableCountries    10158
dtype: int64

In [11]:
df_hbo.dropna(subset=['title'], inplace=True)

In [12]:
#preencher os NaN de genres com "Não informado"
df_hbo['genres'] = df_hbo['genres'].fillna('Não informado')

In [13]:
# os NaN em imdbId e imdbAverageRating podem ser um problema se for analisar a avaliação dos filmes, então preciso ver se o NaN em um coincide com NaN na outra

# Investigar a situação dos NaN
print("Registros sem imdbId:", df_hbo['imdbId'].isna().sum())
print("Registros sem rating:", df_hbo['imdbAverageRating'].isna().sum())

# Ver se NaN em imdbId implica em NaN em rating
mask_sem_id = df_hbo['imdbId'].isna()
print("Dos que não têm ID, quantos também não têm rating:", 
      df_hbo[mask_sem_id]['imdbAverageRating'].isna().sum())

Registros sem imdbId: 359
Registros sem rating: 487
Dos que não têm ID, quantos também não têm rating: 359


In [14]:
# Investigar os casos com ID mas sem avaliação
tem_id_sem_rating = df_hbo['imdbId'].notna() & df_hbo['imdbAverageRating'].isna()
print("Registros com ID mas sem avaliação:", tem_id_sem_rating.sum())

# Ver alguns exemplos desses casos
print("\nExemplos de conteúdo com ID mas sem avaliação:")
exemplos = df_hbo[tem_id_sem_rating][['title', 'imdbId', 'releaseYear']].head()
print(exemplos)

Registros com ID mas sem avaliação: 128

Exemplos de conteúdo com ID mas sem avaliação:
                                      title      imdbId  releaseYear
2831                             Imparables   tt2456834       2014.0
2984                         American Biker   tt4849302       2015.0
3663                              Impromptu  tt27626121       2018.0
3806     Rosa & Dara a jejich dobrodruzství   tt6419668       2015.0
4020  Mimi & Líza: Christmas Lights Mystery  tt10499512       2018.0


In [15]:
#preencher os NaN de imdbid com "Não informado"
df_hbo['imdbId'] = df_hbo['imdbId'].fillna('Não informado')

In [16]:
def impute_ratings_by_genre_decade(df_hbo):
    """
    Imputa valores NaN de imdbAverageRating usando mediana por gênero primário + década
    
    Args:
        df: DataFrame com colunas 'genres', 'releaseYear', 'imdbAverageRating'
    
    Returns:
        DataFrame com ratings imputados e colunas adicionais para rastreamento
    """
    # Extrair gênero primário (primeiro da lista)
    df_hbo['primaryGenre'] = df_hbo['genres'].fillna('Unknown').str.split(',').str[0].str.strip()

    # Extrair década
    df_hbo['decade'] = (df_hbo['releaseYear'] // 10) * 10

    # Flag para indicar se tinha rating original
    df_hbo['hasOriginalRating'] = df_hbo['imdbAverageRating'].notna()

    # Calcular medianas por gênero + década (apenas filmes COM rating)
    films_with_rating = df_hbo[df_hbo['hasOriginalRating']]
    
    medians_genre_decade = (films_with_rating.groupby(['primaryGenre', 'decade'])['imdbAverageRating'].agg(['median', 'count']).reset_index())

    # Calcular medianas só por gênero (fallback)
    medians_genre = (films_with_rating.groupby('primaryGenre')['imdbAverageRating'].median().reset_index())
    
    # Mediana global (último recurso)
    global_median = films_with_rating['imdbAverageRating'].median()

    # Função aninhada para imputar um filme específico
    def impute_single_film(row):
        if row['hasOriginalRating']:
            return pd.Series({
                'imdbAverageRating': row['imdbAverageRating'],
                'ratingImputed': False,
                'imputationSource': 'original'
            })
        
        # Tentar gênero + década primeiro
        genre_decade_match = medians_genre_decade[
            (medians_genre_decade['primaryGenre'] == row['primaryGenre']) & 
            (medians_genre_decade['decade'] == row['decade'])
        ]
        
        if not genre_decade_match.empty:
            median_val = genre_decade_match.iloc[0]['median']
            count = genre_decade_match.iloc[0]['count']
            return pd.Series({
                'imdbAverageRating': median_val,
                'ratingImputed': True,
                'imputationSource': f"{row['primaryGenre']} {int(row['decade'])}s ({count} filmes)"
            })
        
        # Fallback: só gênero
        genre_match = medians_genre[medians_genre['primaryGenre'] == row['primaryGenre']]
        
        if not genre_match.empty:
            median_val = genre_match.iloc[0]['imdbAverageRating']
            return pd.Series({
                'imdbAverageRating': median_val,
                'ratingImputed': True,
                'imputationSource': f"{row['primaryGenre']} geral"
            })
        
        # Último recurso: mediana global
        return pd.Series({
            'imdbAverageRating': global_median,
            'ratingImputed': True,
            'imputationSource': "mediana global"
        })

    # Aplicar imputação
    imputation_results = df_hbo.apply(impute_single_film, axis=1)

    # Combinar resultados
    df_hbo['imdbAverageRating'] = imputation_results['imdbAverageRating']
    df_hbo['ratingImputed'] = imputation_results['ratingImputed']
    df_hbo['imputationSource'] = imputation_results['imputationSource']

    return df_hbo, medians_genre_decade, global_median

In [17]:
# APLICAR A IMPUTAÇÃO
df_imputed, medians_table, global_median = impute_ratings_by_genre_decade(df_hbo)

In [45]:
imputed_count = df_imputed['ratingImputed'].sum()
print(f"Valores imputados: {imputed_count}")

Valores imputados: 487


In [20]:
# Mostrar alguns exemplos organizados
for i, (_, filme) in enumerate(df_imputed.head(8).iterrows()):
    print(f"{i+1:2d}. \"{filme['title']}\" ({int(filme['releaseYear'])})")
    print(f"    Gênero: {filme['primaryGenre']}")
    print(f"    Avaliação imputada: {filme['imdbAverageRating']:.1f}")
    print(f"    Fonte: {filme['imputationSource']}")
    print()

 1. "Kill Bill: Vol. 1" (2003)
    Gênero: Action
    Avaliação imputada: 8.2
    Fonte: original

 2. "Jarhead" (2005)
    Gênero: Biography
    Avaliação imputada: 7.0
    Fonte: original

 3. "Unforgiven" (1992)
    Gênero: Drama
    Avaliação imputada: 8.2
    Fonte: original

 4. "Eternal Sunshine of the Spotless Mind" (2004)
    Gênero: Drama
    Avaliação imputada: 8.3
    Fonte: original

 5. "2001: A Space Odyssey" (1968)
    Gênero: Adventure
    Avaliação imputada: 8.3
    Fonte: original

 6. "Absolute Power" (1997)
    Gênero: Action
    Avaliação imputada: 6.7
    Fonte: original

 7. "Billy Elliot" (2000)
    Gênero: Drama
    Avaliação imputada: 7.7
    Fonte: original

 8. "American History X" (1998)
    Gênero: Crime
    Avaliação imputada: 8.5
    Fonte: original



In [21]:
# Axis 1 para colunas, axis 0 para linhas
df_hbo = df_hbo.drop(['availableCountries'], axis=1)

In [22]:
df_hbo.isnull().sum()

title                  0
type                   0
genres                 0
releaseYear            1
imdbId                 0
imdbAverageRating      0
imdbNumVotes         487
primaryGenre           0
decade                 1
hasOriginalRating      0
ratingImputed          0
imputationSource       0
dtype: int64

In [24]:
# Ver qual é o filme problemático
nulls_year = df_hbo[df_hbo['releaseYear'].isna()]
print(nulls_year[['title', 'type', 'genres', 'imdbId', 'imdbAverageRating']])

                               title type genres      imdbId  \
10062  Untitled One Tree Hill Sequel   tv  Drama  tt33306368   

       imdbAverageRating  
10062               6.75  


In [25]:
# Nem lançado foi ainda...vai de drop
df_hbo = df_hbo.dropna(subset=['releaseYear'])

In [26]:
df_hbo.isnull().sum()

title                  0
type                   0
genres                 0
releaseYear            0
imdbId                 0
imdbAverageRating      0
imdbNumVotes         486
primaryGenre           0
decade                 0
hasOriginalRating      0
ratingImputed          0
imputationSource       0
dtype: int64

In [27]:
#### só depois de remover todos os NaN é que posso converter o releaseYear para int
df_hbo['releaseYear'] = df_hbo['releaseYear'].astype(int)

In [28]:
# ==========================================
# LIMPEZA FINAL - REMOVER BAGAGEM TÉCNICA
# ==========================================

# 1. TRATAR imdbNumVotes para filmes imputados
print("🔧 TRATANDO imdbNumVotes PARA FILMES IMPUTADOS")
print("=" * 50)

# Para filmes com rating imputado, vamos usar a mediana de votos do mesmo gênero/década
films_with_votes = df_hbo[df_hbo['imdbNumVotes'].notna()]

# Calcular mediana de votos por gênero primário
median_votes_by_genre = (films_with_votes
                        .groupby('primaryGenre')['imdbNumVotes']
                        .median()
                        .to_dict())

# Mediana global como fallback
global_median_votes = films_with_votes['imdbNumVotes'].median()

# Função para imputar votos
def impute_votes(row):
    if pd.notna(row['imdbNumVotes']):
        return row['imdbNumVotes']
    
    # Se tem rating imputado, usar mediana do gênero
    if row['ratingImputed']:
        genre_median = median_votes_by_genre.get(row['primaryGenre'])
        if genre_median:
            return genre_median
        else:
            return global_median_votes
    
    return row['imdbNumVotes']



🔧 TRATANDO imdbNumVotes PARA FILMES IMPUTADOS


In [29]:
# Aplicar imputação de votos
df_hbo['imdbNumVotes'] = df_hbo.apply(impute_votes, axis=1)


In [30]:
df_hbo.isnull().sum()

title                0
type                 0
genres               0
releaseYear          0
imdbId               0
imdbAverageRating    0
imdbNumVotes         0
primaryGenre         0
decade               0
hasOriginalRating    0
ratingImputed        0
imputationSource     0
dtype: int64

In [33]:
df_hbo['imdbNumVotes'] = df_hbo['imdbNumVotes'].astype(int)
# Verificar se a conversão foi bem-sucedida
print(df_hbo.dtypes)

title                 object
type                  object
genres                object
releaseYear            int64
imdbId                object
imdbAverageRating    float64
imdbNumVotes           int64
primaryGenre          object
decade               float64
hasOriginalRating       bool
ratingImputed           bool
imputationSource      object
dtype: object


In [34]:
# CRIAR DATASET LIMPO (apenas colunas essenciais) e removendo as colunas criadas para o tratamento
print(f"\n📂 CRIANDO DATASET LIMPO")
print("=" * 30)

colunas_essenciais = [
    'title', 
    'type', 
    'genres', 
    'releaseYear', 
    'imdbId', 
    'imdbAverageRating', 
    'imdbNumVotes'
]

df_hbo_clean = df_hbo[colunas_essenciais].copy()

print(f"Forma final do dataset: {df_hbo_clean.shape}")
print(f"Colunas finais: {list(df_hbo_clean.columns)}")




📂 CRIANDO DATASET LIMPO
Forma final do dataset: (9575, 7)
Colunas finais: ['title', 'type', 'genres', 'releaseYear', 'imdbId', 'imdbAverageRating', 'imdbNumVotes']


In [35]:
# VERIFICAÇÃO FINAL
print(f"\n✅ VERIFICAÇÃO FINAL")
print("=" * 35)

print("Valores nulos por coluna:")
print(df_hbo_clean.isnull().sum())

print(f"\nTipos de dados:")
print(df_hbo_clean.dtypes)


✅ VERIFICAÇÃO FINAL
Valores nulos por coluna:
title                0
type                 0
genres               0
releaseYear          0
imdbId               0
imdbAverageRating    0
imdbNumVotes         0
dtype: int64

Tipos de dados:
title                 object
type                  object
genres                object
releaseYear            int64
imdbId                object
imdbAverageRating    float64
imdbNumVotes           int64
dtype: object


# ======= Exploração Tipo e Gênero=========

In [46]:
tipos_unicos = df_hbo_clean['type'].unique()
print("Tipos únicos na coluna 'type':", tipos_unicos)

Tipos únicos na coluna 'type': ['movie' 'tv']


In [47]:
# Contando quantas produções existem de cada tipo
contagem_tipos = df_hbo_clean['type'].value_counts()
print(f"\nDistribuição por tipo:")
for tipo, quantidade in contagem_tipos.items():
    percentual = (quantidade / len(df_hbo_clean)) * 100
    print(f"{tipo}: {quantidade} produções ({percentual:.1f}%)")


Distribuição por tipo:
movie: 6488 produções (67.8%)
tv: 3087 produções (32.2%)


In [48]:
# EXPLORANDO OS PADRÕES DOS GENRES
print("\nPADRÕES E ESTRUTURA DOS GENRES")
# Amostras aleatórias para entender o formato
print("Exemplos de como os genres aparecem:")
sample_genres = df_hbo_clean['genres'].dropna().sample(10).values
for i, genre in enumerate(sample_genres, 1):
    print(f"{i:2d}. {genre}")


PADRÕES E ESTRUTURA DOS GENRES
Exemplos de como os genres aparecem:
 1. Crime, Documentary
 2. Biography, Documentary, Sport
 3. Reality-TV
 4. Documentary
 5. Horror, Mystery, Thriller
 6. Documentary
 7. Action, Drama, History
 8. Drama, Sci-Fi, Thriller
 9. Comedy
10. Action, Adventure, Sci-Fi


In [49]:
# Função para extrair genres individuais
def extract_individual_genres(df_hbo_clean):
    """Extrai todos os genres individuais de uma coluna que pode conter múltiplos genres"""
    all_genres = []
    
    for genres_str in df_hbo_clean['genres'].dropna():
        # Remove espaços e divide por vírgulas (assumindo que é o separador mais comum)
        if pd.notna(genres_str):
            # Tratamento flexível para diferentes separadores
            genres_list = re.split(r'[,;|]', str(genres_str))
            # Limpa e adiciona à lista
            clean_genres = [genre.strip() for genre in genres_list if genre.strip()]
            all_genres.extend(clean_genres)
    
    return all_genres

In [50]:
# Extraindo todos os genres individuais
individual_genres = extract_individual_genres(df_hbo_clean)

In [51]:
# Contando frequencias
genre_counts = Counter(individual_genres)
print(f"Total de genres únicos encontrados: {len(genre_counts)}")
print(f"Total de ocorrências de genres: {sum(genre_counts.values())}")

Total de genres únicos encontrados: 33
Total de ocorrências de genres: 19368


In [52]:
# TOP GENRES MAIS POPULARES
print("\n TOP 20 GENRES MAIS POPULARES")
top_genres = genre_counts.most_common(20)
for i, (genre, count) in enumerate(top_genres, 1):
    percentage = (count / sum(genre_counts.values())) * 100
    print(f"{i:2d}. {genre:<25} | {count:>6} ocorrências ({percentage:>5.2f}%)")



 TOP 20 GENRES MAIS POPULARES
 1. Drama                     |   3502 ocorrências (18.08%)
 2. Comedy                    |   2479 ocorrências (12.80%)
 3. Documentary               |   1937 ocorrências (10.00%)
 4. Crime                     |   1268 ocorrências ( 6.55%)
 5. Adventure                 |   1166 ocorrências ( 6.02%)
 6. Action                    |   1135 ocorrências ( 5.86%)
 7. Animation                 |    918 ocorrências ( 4.74%)
 8. Romance                   |    895 ocorrências ( 4.62%)
 9. Reality-TV                |    889 ocorrências ( 4.59%)
10. Thriller                  |    770 ocorrências ( 3.98%)
11. Mystery                   |    561 ocorrências ( 2.90%)
12. Biography                 |    533 ocorrências ( 2.75%)
13. Family                    |    495 ocorrências ( 2.56%)
14. Horror                    |    460 ocorrências ( 2.38%)
15. Fantasy                   |    403 ocorrências ( 2.08%)
16. History                   |    363 ocorrências ( 1.87%)
17. Sci-F

In [44]:
# Salva o DataFrame limpo
df_hbo_clean.to_csv("data_hbo_clean.csv", index=False)